In [4]:
import os 

class HMM:
    #parameter T?
    y_star=[]
    #y_star={}
    def __init__(self,x,y):
        self.x = x # x are words
        self.y = y # y are tags
        
        
    #part2
    def count_y(self,y_i):
        
        count = 0
        for i in self.y:
            if y_i == i:
                count+=1
        return count
    
    
    #2a count(y pair x)/ count(y)
    def est_emission(self,y_i,x_i):
        
        count = 0
        for i in range(len(self.y)): 
            if y_i == self.y[i] and x_i == self.x[i]:
                count +=1
        return count/self.count_y(y_i)
    
    
    #2b
    def impr_est_emission(self,y_i,x_i):
        
        if x_i not in self.x:
            return 1/(self.count_y(y_i)+1)
        
        else:
            count = 0
            for i in range(len(self.y)): 
                if y_i == self.y[i] and x_i == self.x[i]:
                    count +=1
            return count/(self.count_y(y_i)+1)
    

    #2c should y_star be a dictionary so can be referenced with new data set?
    def sentiment_analysis(self,data_x):
        y_star=self.y_star
        for i in range(len(data_x)):
            em_list = []
            for j in range(len(self.y)):
                emission = self.impr_est_emission(self.y[j],data_x[i])
                em_list.append(emission)
            y_arg = em_list.index(max(em_list))
            y_star.append(self.y[y_arg])
            #y_star[data_x[i]]=self.y[y_arg]
        return y_star

    
    

# format: [[words], [tags]]
def read_train(file_name):
    in_file = open(file_name,'r')
    l = []
    words = []
    tags = []
    for line in in_file:
        x = line.strip().split()
        if x != []:
            words.append(x[0])
            tags.append(x[1].rstrip('\n'))
    l.append(words)
    l.append(tags)
    return l
    
#reading and writing to files    
def read_dev_in(file_name):
    in_file = open(file_name,'r')
    l = []
    for line in in_file:
        l.append(line.strip())
    return l
      
def read_dev_out(file_name):
    l = []
    in_file = open(file_name,'r')
    for line in in_file:
        x = line.strip()
        l.append(x.rstrip('\n').split())
    #print (l)
    l2 = [x for x in l if x != []]
    return l2
# format: [['word', 'tag'], ['word', 'tag]...]

# gold-stndard outputs in dev.out
def entities(l):
    d = {}
    word = ""
    for i in range(len(l)):
        if l[i][1] != "O":
            word += l[i][0]
            tag = l[i][1][2:]
            if ("I-" + tag) not in l[i+1][1]:
                d[word.lstrip()] = tag
                word = ""
            word += " "
    return d
    
#l = read_dev_out("SG/dev.out")
#print (entities(l))

#print (read_dev_in("SG/dev.in"))


#My initial test code for HMM class basic functions
'''test_x = ["bad","good","good","hello"]
test_y = ["-","+","+","+"]
test_datax = ["bad","day","hello"]
test_HMM = HMM(test_x,test_y)
print(test_HMM.count_y("+"))
print(test_HMM.impr_est_emission("-","day"))
print(test_HMM.sentiment_analysis(test_datax))'''


data = read_train("SG/train")
SG_HMM = HMM(data[0], data[1]) # x and y from train
print (len(data[1]))
datax = read_dev_in("SG/dev.in")
print (len(datax))
print (SG_HMM.sentiment_analysis(datax))

120156
51071


KeyboardInterrupt: 